In [10]:
using FileIO
using CSV
using XLSX
using DataFrames
using KnetMetrics
using RCall

In [11]:
#Import data as excel:
#dirpath = "/mnt/s/OneDrive - Universiteit Leiden/QSTDN_Quantum&Society/data/national_newspapers_09-21/" #pilot_coding_data/"
dirpath = "/mnt/s/OneDrive - Universiteit Leiden/"
filename = "Coding_Sheet_InterCoder_medium.xlsx" #"Pilot_Coding_Sheet_InterCoder.xlsx"
sheetname = "IC_Parallel" # "Pilot_IC_Parallel"
dataloc_on_sheet = "A7:GC85" #"A7:GC43" #Must include header row!!

#df = DataFrame(XLSX.readtable(dirpath*filename, sheetname))
xlf_mat = XLSX.readdata(dirpath*filename, sheetname*"!"*dataloc_on_sheet)

colnames = xlf_mat[1,:]
data = xlf_mat[2:end,:]
header = Symbol.(colnames[:])

data[.|(data .== nothing, string.(data) .== "-2", data .== "~")] .= missing

#data[data .== "~"] .= missing
#@show data

df = DataFrame(data, header)
@show header

header = [Symbol("Article #"), Symbol("1"), Symbol("2"), Symbol("3"), Symbol("4"), Symbol("5"), Symbol("6"), Symbol("7"), Symbol("8"), Symbol("9"), Symbol("10"), Symbol("A.11.A"), Symbol("A.12"), Symbol("A.13"), Symbol("A.14"), Symbol("A.14-Q"), Symbol("A.15"), Symbol("A.16.A"), Symbol("A.16.B"), Symbol("A.16.C"), Symbol("A.16.D"), Symbol("A.16.E"), Symbol("A.16.F"), Symbol("A.16.G"), Symbol("A.16.H"), Symbol("A.16.I"), Symbol("A.16-Q"), Symbol("A.17"), Symbol("A.17-Q"), Symbol("A.18"), Symbol("A.18-Q"), Symbol("A.19"), Symbol("A.19-Q"), Symbol("A.20"), Symbol("A.21"), Symbol("A.21-Q"), Symbol("A.22"), Symbol("A.23.A"), Symbol("A.23.B"), Symbol("A.23.C"), Symbol("A.23.D"), Symbol("A.23.E"), Symbol("A.23.F"), Symbol("A.23.G"), Symbol("A.24"), Symbol("A.24-Q"), Symbol("A.25"), Symbol("A.26.A"), Symbol("A.26.B"), Symbol("A.26.C"), Symbol("A.26.D"), Symbol("A.26.E"), Symbol("A.26.F"), Symbol("A.26.G"), Symbol("A.27"), Symbol("A.28.A"), Symbol("A.28.B"), Symbol("A.28.C"), Symbol("A.28.D"), 

185-element Array{Symbol,1}:
 Symbol("Article #")
 Symbol("1")
 Symbol("2")
 Symbol("3")
 Symbol("4")
 Symbol("5")
 Symbol("6")
 Symbol("7")
 Symbol("8")
 Symbol("9")
 ⋮
 Symbol("ICA.29.A")
 Symbol("ICA.29.B")
 Symbol("ICA.29.C")
 Symbol("ICA.30.A")
 Symbol("ICA.30.B")
 Symbol("ICA.30.C")
 Symbol("ICA.30-Q-A")
 Symbol("ICA.30-Q-B")
 Symbol("ICA.30-Q-C")

In [12]:
#package1 = "irr"
#@rput package1
#R"install.packages(package1)"
#package2 = "psych"
#@rput package2
#R"install.packages(package2)"

R"library(irr)"
R"library(psych)"

dinfo1 = nothing
dinfo2 = nothing
dinfo3 = nothing
dinfo4 = []

krippalpha_outputs = []
cohenkappa_outputs = []
percentagree_outputs = []
count_outputs = []
agreementCount_outputs = []

A_cols = filter(cn -> startswith(String(cn), "A."), header)
B_cols = filter(cn -> startswith(String(cn), "B."), header)
var_types = fill("nominal", 58) #Column 11 to 40 only, including subcolumns = 58

for (col_a, col_b, c_var_type) in zip(A_cols, B_cols, var_types)
    c_var_data_df = df[:, [col_a, col_b]]
    
    c_var_data = Matrix(c_var_data_df)
    #kripp_alpha(c_var_data, method = c_var_type)
    @rput c_var_data
    @rput c_var_type
    #dinfo1 = c_var_data
    R"c_var_data <- t(matrix(unlist(c_var_data), byrow=FALSE, ncol=2))"
    #R"dinfo2 <- c_var_data"
    #@rget dinfo2

    R"krippalpha_output <- kripp.alpha(c_var_data, method = c_var_type)"
    @rget krippalpha_output
    push!(krippalpha_outputs, krippalpha_output)
    
    c_var_data = c_var_data_df[completecases(c_var_data_df), :] #Remove missin balues

    percent_agree = 100 * count(c_var_data[:, 1] .== c_var_data[:, 2]) / size(c_var_data)[1]
    push!(count_outputs,  size(c_var_data)[1])
    push!(agreementCount_outputs, count(c_var_data[:, 1] .== c_var_data[:, 2]))
    push!(percentagree_outputs, percent_agree)
        
    if prod(size(c_var_data)) == 0 #If all cases missing
        cohenkappa_output = Dict(:value=>missing)
        push!(cohenkappa_outputs, cohenkappa_output)
    else
        #push!(dinfo4, size(c_var_data))
        @rput c_var_data
        R"c_var_data <- t(matrix(unlist(c_var_data), byrow=FALSE, ncol=2))"
        weight_mode = "unweighted"
        @rput weight_mode

        R"cohenkappa_output <- kappa2(t(c_var_data), weight = weight_mode, sort.levels = FALSE)"
        #R"cohenkappa_output <- cohen.kappa(as.data.frame(t(c_var_data)))"
        @rget cohenkappa_output
        push!(cohenkappa_outputs, cohenkappa_output)
        
    end
    #View confusion matrix
    #c = confusion_matrix(c_var_data[:, 1],c_var_data[:, 2], labels=[0,1])
    

end

┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172


┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home/trothe/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning in kripp.alpha(c_var_data, method = c_var_type) :
│   NAs introduced by coercion
└ @ RCall /home

In [13]:
@show cohenkappa_outputs

cohenkappa_outputs = Any[OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.6172520661157025,:stat_name => "z",:statistic => 7.594842718636398,:p_value => 3.086420008457935e-14), OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.7817164179104477,:stat_name => "z",:statistic => 10.866314095352433,:p_value => 0.0), OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.6608695652173929,:stat_name => "z",:statistic => 6.204313539264316,:p_value => 5.493616672680446e-10), OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.948616600790513

58-element Array{Any,1}:
 OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.6172520661157025,:stat_name => "z",:statistic => 7.594842718636398,:p_value => 3.086420008457935e-14)
 OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.7817164179104477,:stat_name => "z",:statistic => 10.866314095352433,:p_value => 0.0)
 OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.6608695652173929,:stat_name => "z",:statistic => 6.204313539264316,:p_value => 5.493616672680446e-10)
 OrderedCollections.OrderedDict{Symbol,Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)",:subjects => 78,:raters => 2,:irr_name => "Kappa",:value => 0.94861660079051

In [14]:
#Compile outputs to simple dataframe:

code_numbers = replace.(String.(A_cols), "A." => "") 

irr_output_df = DataFrame(CodingPoint=code_numbers,
        Count = count_outputs,
        AgreementCount = agreementCount_outputs,
        PercentAgreeement=percentagree_outputs, 
        CohenKappa=[output[:value] for output in cohenkappa_outputs] , 
        KrippAlpha=[output[:value] for output in krippalpha_outputs])

XLSX.writetable(dirpath*"output_table.xlsx", irr_output_df,  anchor_cell="B2")
